In [270]:
import os, sys, codecs, time, datetime
import fileinput
import os.path
import stanza
from io import StringIO
from subprocess import call


from docopt import docopt
from propsde.applications.viz_tree import DepTreeVisualizer
%load_ext autoreload
%autoreload 2
import propsde.applications.run as run
import sys
if sys.version_info[0] >= 3:
    unicode = str
    
stanza.download('de')
nlp = stanza.Pipeline('de')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


2020-05-15 11:53:23 INFO: Downloading default packages for language: de (German)...
2020-05-15 11:53:24 INFO: File exists: /home/chris/stanza_resources/de/default.zip.
2020-05-15 11:53:29 INFO: Finished downloading models and saved to /home/chris/stanza_resources.
2020-05-15 11:53:29 INFO: Loading these models for language: de (German):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| pos       | gsd     |
| lemma     | gsd     |
| depparse  | gsd     |
| ner       | conll03 |

2020-05-15 11:53:29 INFO: Use device: gpu
2020-05-15 11:53:29 INFO: Loading: tokenize
2020-05-15 11:53:29 INFO: Loading: mwt
2020-05-15 11:53:29 INFO: Loading: pos
2020-05-15 11:53:30 INFO: Loading: lemma
2020-05-15 11:53:30 INFO: Loading: depparse
2020-05-15 11:53:30 INFO: Loading: ner
2020-05-15 11:53:31 INFO: Done loading processors!


In [271]:
import pickle

In [272]:
with open('polizeiberichte.pkl', 'rb') as f:
    data = pickle.load(f, encoding="utf8")

Gute Texte: 5
Schwierige Texte: 11

In [273]:
text_example = data[80]["text1"]
print(text_example)

Gestern Nachmittag kam es zu einem schweren Verkehrsunfall in Rudow. Gegen 17.30 Uhr fuhr ein „Opel“ auf der Schönefelder Straße in Richtung Rudower Spinne. Als die Fahrerin den ersten Ermittlungen zufolge die Kreuzung Neuköllner Straße passieren wollte, übersah sie offenbar einen kreuzenden „Mercedes“, der auf der Neuköllner Straße in Richtung Waltersdorfer Chaussee unterwegs war. Es kam zu einem Zusammenstoß. Der „Mercedes“ geriet ins Schleudern, kippte auf die Fahrerseite und prallte gegen einen geparkten „Seat“. Die Feuerwehr musste den 49 Jahre alten Fahrer aus seinem Fahrzeug bergen und brachte ihn zur stationären Behandlung in ein Krankenhaus. Die 42-jährige „Opel“-Fahrerin blieb unverletzt.


In [546]:
file = codecs.open("examples.txt", encoding='utf8')
gs = run.parseSentences(file)

running mate-tools
lemmatizing
parsing + tagging
collapsing
java -jar ext/org.jobimtext.collapsing.jar -i tmp/parsed.conll06 -o tmp -sf -l de -r ext/resources/german_modified.txt -f c -np -nt
UFF


In [307]:
for graph, tree in gs:
    original_line = str(graph).split("\n")[0].split(" ")

In [9]:
prop = g.getPropositions('pdf')[0]
print(prop.args)
print(prop.pred)

[['prop_of', 'Die 42-jährige Opel-Fahrerin ']]
unverletzt


In [10]:
test = gs[0][0]
print(test)

Gestern Nachmittag kam es zu einem schweren Verkehrsunfall in Rudow .
0	1,Gestern	ADV	0	0	mod,2
1	2,Nachmittag	ADV	0	0	mod,2
2	3,kam	VVFIN	1	1	
3	4,es	PPER	0	0	dep,2
5	7,schweren	ADJA	0	0	mod,6
6	8,Verkehrsunfall	NN	0	0	prep_zu,2
8	10,Rudow	NE	0	0	prep_in,6



# ParZU Dependency Parser

In [37]:
import urllib.request
import urllib.parse

def parse_dependencies_to_conll_file(text, path):
    params = {
        "text": text,
        "format": "conll"
    }
    args = urllib.parse.urlencode(params).encode("utf-8")
    req = "http://localhost:5003/parse/?"+args.decode("utf-8")
    try:
        f = urllib.request.urlopen(req)
        file = open(path,'w')
        file.write(f.read().decode('utf-8'))
        return 0
    except Exception as ex:
        print(ex)
        return 1
    


# CorZu 2.0

In [547]:
import os
output = "corzu_export"
conll = "test.conll"
markables = "markables"
text = open("examples.txt", "r").read()

err = parse_dependencies_to_conll_file(text, conll)
if err == 0:
    cmd = "python ext/CorZu_v2.0/extract_mables_from_parzu.py " + conll + " > " + markables
    err = os.system(cmd)
    print(err)



0


In [548]:
cmd = "python ext/CorZu_v2.0/corzu.py " + markables + " " + conll + " " + output    
err = os.system(cmd)
print(err)

0


# Work with CorZu Extraction

In [550]:
import re

def find_np(lines, id):
    np = ""
    indices = 0
    for line in lines:
        line_list = re.split('\t| +',line)
        np += line_list[1] + " "
        indices += 1
        if line.strip().endswith(id + ')'):
            return np, indices


text = open(output, "r").read()
lines=text.split('\n\n')

for i in range(0, len(lines)):
    lines[i] = lines[i].split("\n")
    for j in range(0, len(lines[i])):
        lines[i][j] = lines[i][j].split("\t")
lines = {i: l for i, l in enumerate(lines) if len(l) > 0}


for sentence_count, gt in enumerate(gs):
    graph, tree = gt
    word_ids = []
    for node in graph.nodes():
        index = int(node.to_conll_like().split("\t")[1].split(",")[0]) 
        if lines[sentence_count][index][-1] != "-" and lines[sentence_count][index][-1] != "":
            print(node.get_original_text())
            node.features["coreference"] = lines[sentence_count][index][-1]


KeyError: <propsde.graph_representation.newNode.Node object at 0x7ff98c6e1208>

In [541]:
for graph, tree in gs:
    for node in graph:
        if "coreference" in node.features:
            print(node.text[0])
            print(node.features["coreference"])

Kreuzung
(0
Straße
0)
passieren
0)
sie
(0)
kreuzenden
(1
Mercedes
1)
Mercedes
1)
geriet
1)
seinem
(2)
Fahrzeug
2)
bergen
2)
ihn
(2)
49
(2
haben
(2)


In [ ]:
sentence_index = 0
for graph, tree in gs:
    sentences[sentence_index] = {}
    for line in lines:
            l = line.split('\t')
            line_list = re.split('\t| +',line)

            id = re.findall('\d+',line_list[-1])
            if id and id[0] not in entities and line.strip().endswith('(' + id[0]):
    
                sentences[sentence_index][l[0]] = id[0]
                print(i, l[0], l[1])
                entities[i] = "(" + id[0]
    
                for j in range(1, indices-2):
         
                    entities[i + j] = "-"
                entities[i + indices] = id[0] + ")"
    
            elif id and line.strip().endswith('(' + id[0] + ')'):
                entities[i] = "(" + id[0] + ")"
    sentence_index += 1

In [313]:
print(entities)
print(conll_text[67])

{33: '(0', 34: '-', 37: '0)', 41: '(0)', 43: '(1', 46: '1)', 65: '(1', 67: '1)', 85: '(2', 86: '-', 87: '-', 88: '-', 89: '-', 92: '2)', 91: '(2)', 96: '(2)'}
geriet


# Combining CorZu and PropsDE

### Extract propositions with words and their ids

In [256]:
from functools import reduce
from itertools import product
from propsde.graph_representation.graph_utils import get_min_max_span, find_nodes, \
    find_edges, merge_nodes, multi_get, duplicateEdge, accessibility_wo_self, \
    replace_head, find_marker_idx
from propsde.utils.utils import encode_german_characters, encode_german_chars
from propsde.graph_representation.proposition import Proposition
from propsde.graph_representation.word import NO_INDEX, Word, strip_punctuations

def subgraph_to_string(graph,node,exclude=[]):
    nodes = [node]
    change = True
    while change:
        change=False
        for curNode in nodes:
            for curNeigbour in graph.neighbors(curNode):
                if (curNeigbour in nodes) or (curNeigbour in exclude): continue
                nodes.append(curNeigbour)
                change = True
    #print [[w.word for w in x.text] for x in nodes]

#     minInd = min([n.minIndex() for n in nodes])-1
#     maxInd = max([n.maxIndex() for n in nodes])-1
#     ret = " ".join(graph.originalSentence.split(" ")[minInd:maxInd+1])
#     nodes = [n for n in minimal_spanning_tree(graph, node) if not n in exclude]
#     ret = " ".join(node.get_original_text() for node in sorted(nodes,key = lambda n: n.minIndex()))
    try:
        ret = ""
        words = []
        ids = []
        for n in nodes:
            words += n.surface_form
            ids.append(n.uid)
            # add collapsed prepositions
            for parent_rel in n.incidents():
                # but just nested ones
                if parent_rel.startswith("prep_") and n.incidents()[parent_rel][0] in nodes:
                    idx = get_min_max_span(graph,n)[0] - 1
                    w = Word(idx,parent_rel.replace("prep_",""))
                    words += [w]    

        words = list(set(words))
        for w,i  in zip(words, ids):
            print(w, i)
        ret = " ".join([w.word for w in strip_punctuations(sorted(words,key=lambda w:w.index))])+" "

    except:
        raise Exception()
#     
    
    return "schwub"

def getPropositions(self):
    ret = []
    for topNode in [n for n in self.nodes() if n.features.get("top", False)]:
            #print [w.word for w in topNode.text]
#            if "dups" in topNode.features:
            dups = topNode.features.get("dups", [])
            allDups = reduce(lambda x, y:list(x) + list(y), dups, [])

            rest = [n for n in self.neighbors(topNode) if n not in allDups]
            neigboursList = []
            for combination in product(*dups):
                curNeigbourList = []
                ls = list(combination) + rest
                for curNode in ls:
                    curNeigbourList.append((self.edge_label((topNode, curNode)), curNode))
                neigboursList.append(curNeigbourList)
         
            for nlist in neigboursList:
                #print [[w.word for w in x[1].text] for x in nlist]
                argList = []
                all_neighbours = [n for _, n in nlist]
                for k, curNeighbour in sorted(nlist, key=lambda k_n:get_min_max_span(self, k_n[1])[0]):
                    curExclude = [n for n in all_neighbours if n != curNeighbour] + [topNode]
                    if self.edge_label((topNode,curNeighbour)) != "dep":
                        
                        argList.append([k, subgraph_to_string(self, curNeighbour, exclude=curExclude), curNeighbour.uid])
   
                if topNode.features.get("Lemma"):
                    topNodeText = encode_german_characters(topNode.features.get("Lemma"))
                else:
                    topNodeText = topNode.get_original_text()
                if topNode.features.get("Negation",False):
                    topNodeText = "nicht " + topNodeText
                argList = [a for a in argList if not a[1].strip() in ["und","oder"]]
                
                if not len(argList) == 0:
                    ret.append((topNode, argList))
    return ret


In [285]:
word_indices = {}
ready = {}
current_length = 0
predicates = []

for g, tree in gs:     
    prop = getPropositions(g)
    print(type(tree))
    print(tree)
    for arg in prop[0][1]:
        try:
            print(entities[arg[-1]])
            print(arg)
        except:
            pass

Gestern 669
Nachmittag 670
Verkehrsunfall 675
in 677
einem 674
<class 'str'>
1	Gestern	_	ADV	ADV	_	3	MO	_	_
2	Nachmittag	_	ADV	ADV	_	3	MO	_	_
3	kam	_	VVFIN	VVFIN	_	0	--	_	_
4	es	_	PPER	PPER	_	3	EP	_	_
6	einem	_	ART	ART	_	8	NK	_	_
7	schweren	_	ADJA	ADJA	_	8	NK	_	_
8	Verkehrsunfall	_	NN	NN	_	3	MO_zu	_	_
9	in	_	APPR	APPR	_	8	MNR	_	_
10	Rudow	_	NE	NE	_	9	NK	_	_
11	.	_	$.	$.	_	10	--	_	_

17.30 773
ein 684
Spinne 687
der 689
in 686
. 774
<class 'str'>
2	17.30	_	CARD	CARD	_	3	NK	_	_
3	Uhr	_	NN	NN	_	4	MO_gegen	_	_
4	fuhr	_	VVFIN	VVFIN	_	0	--	_	_
5	ein	_	ART	ART	_	6	NK	_	_
6	Opel	_	NN	NN	_	4	SB	_	_
8	der	_	ART	ART	_	10	NK	_	_
9	Schönefelder	_	ADJA	ADJA	_	10	NK	_	_
10	Straße	_	NN	NN	_	4	MO_auf	_	_
11	in	_	APPR	APPR	_	10	MNR	_	_
12	Richtung	_	NN	NN	_	11	NK	_	_
13	Rudower	_	NE	NE	_	14	PNC	_	_
14	Spinne	_	NE	NE	_	12	NK	_	_
15	.	_	$.	$.	_	14	--	_	_

, 713
, 713
der 718
Waltersdorfer 720
in 717
Straße 775
unterwegs 723
Kreuzung 705
, 704
die 694
die 696
passieren 702
Straße 703
den 700
zufolge 699
Er

In [20]:
from itertools import compress

def _extract_subj_from_article_stanza(pos_tag, personal_pronouns, pronoun):
    for sent in pos_tag.sentences:
        for word in sent.words:
            if word.deprel == "nsubj" or word.deprel == "root":
                if callable(personal_pronouns[pronoun]):
                    _pronoun = personal_pronouns[pronoun](word.text)
                else:
                    _pronoun = personal_pronouns[pronoun]
                return (_pronoun, word.lemma)
            
def _get_pronoun_for_den(word):
    if word[-2:] == "en":
        return "sie"
    else:
        return "er"

def _map_article_of_subj_to_pronoun(arg):
    personal_pronouns = {"der": "er", "das": "es", "die": "sie", "dem": "er", "den": _get_pronoun_for_den}
    
    if "subj" in arg:
        words = arg[1].lower().split()
        pronoun_place = list(word in personal_pronouns for word in words)
        pronoun = list(compress(words, pronoun_place))
    
        if len(pronoun) == 1:
            pos_tag = nlp(arg[1])
            return _extract_subj_from_article_stanza(pos_tag, personal_pronouns, pronoun[0])
    
    for pronoun in personal_pronouns:
        if pronoun in arg[1] and "prep" not in arg[0]:
            pos_tag = nlp(arg[1])
            return _extract_subj_from_article_stanza(pos_tag, personal_pronouns, pronoun)
                        
    return None

def _get_core_from_noun_phrase(noun_phrase):
    pos_tag = nlp(noun_phrase)
    for sent in pos_tag.sentences:
        for word in sent.words:
            if word.head == 0:
                return {"np": noun_phrase, "core":word.lemma}
            
def _get_context(prop_cache, subjects, words):
    for j in range(0, len(subjects)):
        for k in range(len(subjects[index-j]) - 1, 0, -1):
            if index > 0 and len(subjects[index-j]) > 0 and subjects[index-j][k][0] in words:
                subject_index = words.index(subjects[index-j][k][0])
                return _get_core_from_noun_phrase(subjects[index-j][k][1])["core"]
            
def pretty_print(prop_cache):
    for prop in prop_cache:
        test = {}
        for arg in prop["args"]:
            if "context" in arg[1]:
                test[arg[0]] = arg[1]["context"]
            else:
                test[arg[0]] = arg[1]["core"]
        prep_exists = False
        for k in test:
            if "_" in k:
                if "conj" in k:
                    pass
                else:
                    if "subj" in k:
                        print(test["subj"] + " " + prop["pred"] + " " + k + " " + test[k])
                prep_exists = True
        if not prep_exists:  
            print(test["subj"] + " " + prop["pred"] + " " + test["dobj"])
        
            



In [90]:
subjects = []
prop_cache = []
index = 0

for i, (g, tree) in enumerate(gs):
    for prop in g.getPropositions('pdf'):
        subjects.append([])
        prop_cache.append({"args": prop.args, "pred": prop.pred})
        
while index < len(prop_cache):
    for i, arg in enumerate(prop_cache[index]["args"]):
        words = arg[1].lower().split()

        _article_pronoun = _map_article_of_subj_to_pronoun(arg)
        _noun_phrase_core = _get_core_from_noun_phrase(arg[1])
        if _article_pronoun:
            subjects[index].append(_article_pronoun)
        if _noun_phrase_core:
            prop_cache[index]["args"][i][1] = _noun_phrase_core
        context = _get_context(prop_cache, subjects, words)
        if context:
            prop_cache[index]["args"][i][1]["context"] = context

                    

    index += 1
pretty_print(prop_cache)
print(subjects)

Ermittlung offenbar übersehen Mercedes
Fahrer haben Fahrzeug
[[], [], [], [], [('sie', 'Fahrerin'), ('sie', 'Ermittlung')], [], [('er', 'Mercedes')], [('sie', 'Feuerwehr'), ('er', 'Fahrer')], [('sie', 'Feuerwehr')], [], [('er', 'Feuerwehr')], []]


Der regierende Bürgermeister fährt zum Rathaus.
Im roten Rathaus geht er in den Sitzungssaal.
Der alte Sitzungssaal liegt neben dem Büro von Michael Müller.

In [503]:
for prop in prop_cache:
    str_build = ""
    
    for arg in prop["args"]:
        if arg[0] is not "mod":
            if "context" in arg[1]:
                str_build += arg[1]['context'] + "/"

            str_build += arg[1]['core'] + "/"
    print(prop["pred"] + ": " + str_build)

kommen: Verkehrsunfall/
fahren: Uhr/Opel/Straße/
kreuzenden: Mercedes/
sein: Mercedes/Straße/
offenbar übersehen: Ermittlung/sie/Mercedes/
kommen: Zusammenstoß/
geraten: Mercedes/schleudern/
mussen: Feuerwehr/Fahrer/Fahrzeug/
bringen: Feuerwehr/er/Behandlung/Krankenhaus/
haben: Fahrer/er/Fahrzeug/
und: bergen/bringen/
unverletzt: Opel/


versuchen:(subj:Die Täter , prep_in:der Nacht vom 19 )
Juni 2014:(dobj:die Eingangstür des Bürokomplexes in der Feldtmannstraße gewaltsam zu öffnen )
gelangt:(subj:sie , prep_in:das Objekt , mod:so )
aufhebeln:(subj:sie , dobj:ein Fenster an der Rückseite des Hauses , mod:Als dies nicht gelang , mod:schließlich )
und:(conj_und:Als dies nicht gelang , hebelten sie schließlich ein Fenster an der Rückseite des Hauses auf , conj_und:sie gelangten so in das Objekt )
entwenden:(subj:sie , dobj:auch eine EC-Karte , die bereits an Freitag , 20 , prep_aus:den Büroräumen mehrerer dort ansässiger Firmen , prep_neben:Geld )
20:(subj:auch eine EC-Karte , prep_an:Freitag , mod:bereits )
einsetzen:(obj:Juni 2014 , prep_gegen:5.10 Uhr , prep_an:einem Geldautomaten in der Danziger Straße )

In [504]:
for prop in prop_cache:
    for arg in prop["args"]:
        print(arg[1])

{'np': 'Gestern ', 'core': 'Gestern'}
{'np': 'Nachmittag ', 'core': 'Nachmittag'}
{'np': 'einem schweren Verkehrsunfall in Rudow ', 'core': 'Verkehrsunfall'}
{'np': '17:30 Uhr ', 'core': 'Uhr'}
{'np': 'ein Opel ', 'core': 'Opel'}
{'np': 'der Schönefelder Straße in Richtung Rudower Spinne ', 'core': 'Straße'}
{'np': 'einen Mercedes ', 'core': 'Mercedes'}
{'np': 'einen Mercedes ', 'core': 'Mercedes'}
{'np': 'der Neuköllner Straße in Richtung Waltersdorfer Chaussee ', 'core': 'Straße'}
{'np': 'unterwegs ', 'core': 'unterwegs'}
{'np': 'Als die Fahrerin die Kreuzung Neuköllner Straße passieren wollte ', 'core': 'passieren'}
{'np': 'den ersten Ermittlungen zufolge ', 'core': 'Ermittlung'}
{'np': 'sie ', 'core': 'sie', 'context': 'Ermittlung'}
{'np': 'einen Mercedes ', 'core': 'Mercedes'}
{'np': 'einem Zusammenstoß ', 'core': 'Zusammenstoß'}
{'np': 'Der Mercedes ', 'core': 'Mercedes'}
{'np': 'Schleudern ', 'core': 'schleudern'}
{'np': 'Die Feuerwehr ', 'core': 'Feuerwehr'}
{'np': 'den 49 Jahr

In [494]:
nlp('Die Feuerwehr brachte ihn zu stationären Behandlung in ein Krankenhaus ')

[
  [
    {
      "id": "1",
      "text": "Die",
      "lemma": "der",
      "upos": "DET",
      "xpos": "ART",
      "feats": "Case=Nom|Definite=Def|Gender=Fem|Number=Sing|PronType=Art",
      "head": 2,
      "deprel": "det",
      "misc": "start_char=0|end_char=3"
    },
    {
      "id": "2",
      "text": "Feuerwehr",
      "lemma": "Feuerwehr",
      "upos": "NOUN",
      "xpos": "NN",
      "feats": "Case=Nom|Gender=Fem|Number=Sing",
      "head": 3,
      "deprel": "nsubj",
      "misc": "start_char=4|end_char=13"
    },
    {
      "id": "3",
      "text": "brachte",
      "lemma": "bringen",
      "upos": "VERB",
      "xpos": "VVFIN",
      "feats": "Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin",
      "head": 0,
      "deprel": "root",
      "misc": "start_char=14|end_char=21"
    },
    {
      "id": "4",
      "text": "ihn",
      "lemma": "er",
      "upos": "PRON",
      "xpos": "PPER",
      "feats": "Case=Acc|Gender=Masc|Number=Sing|Person=3|PronType=Prs",


1	was	was	PRO	PIS	_|Nom|Sg	2	subj	_	_
2	geht	gehen	V	VVFIN	3|Sg|Pres|Ind	0	root	_	_
3	ab	ab	PTKVZ	PTKVZ	_	2	avz	_	_
4	.	.	$.	$.	_	0	root	_	_


